In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/MyDrive/Proj

/content/gdrive/MyDrive/Proj


Importing required libraries

In [3]:
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from tqdm import tqdm
import copy
import time

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms
from torchvision import models
from torch import optim, nn
import torch.nn.functional as F

In [4]:
device = 'cpu'

if torch.cuda.is_available() :
    device = 'cuda'

Creating dataset

In [5]:
class ImageDataset(Dataset) :

    def __init__(self, transform) :
        self.root_path = 'PACS/kfold/'

        # Listing the domains
        self.domains = os.listdir(self.root_path)

        # Listing the classes 
        self.classes = os.listdir(self.root_path+'cartoon')

        # Transformations
        self.transforms = transform

        self.images = []
        self.domains_y = []
        self.classes_y = []

        for i_dom, domain in enumerate(self.domains) :
            for i_cla, cla in enumerate(self.classes) :
                for image in os.listdir(self.root_path+domain+'/'+cla) :
                    # Finding image path
                    image_path = self.root_path+domain+'/'+cla+'/'+image
                    img = cv2.imread(image_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    self.images.append(img)

                    # One hot encoding domain
                    domainVector = np.zeros(5)
                    domainVector[-1] = 1
                    domainVector[i_dom] = 1
                    self.domains_y.append(domainVector)

                    # One hot encoding class
                    classVector = np.zeros(7)
                    classVector[i_cla] = 1
                    self.classes_y.append(classVector)

        self.images = np.array(self.images)
        self.domains_y = np.array(self.domains_y)
        self.classes_y = np.array(self.classes_y)

        self.domains_y = torch.Tensor(self.domains_y)
        self.classes_y = torch.Tensor(self.classes_y)

    def __getitem__(self, index) :

        return self.transforms(self.images[index].astype('float')/255), self.domains_y[index], self.classes_y[index]

    def __len__(self) :
        return len(self.images)

Defining transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(50),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [7]:
dataset = ImageDataset(transform=transform)

In [8]:
# Train and test split
train_dataset, val_dataset, test_dataset = random_split(dataset, [6000, 1000, 2991])

In [9]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=True)

In [10]:
dataloaders = {
    'train' : train_dataloader,
    'val' : val_dataloader,
    'test' : test_dataloader
}

dataset_sizes = {
    'train' : 6000,
    'val': 1000,
    'test' : 2991
}

In [11]:
def getWeightsAndBiases(in_size, out_size, no_domains) :

    weight_mat = torch.zeros(out_size, in_size, no_domains+1)
    bias_mat = torch.zeros(out_size, no_domains+1)

    for i in range(no_domains+1) :
        linMod = nn.Linear(in_size, out_size)

        weight = linMod.weight.detach()
        bias = linMod.bias.detach()

        if not(i == no_domains) :
          weight_mat[:, :, i] = weight/no_domains
          bias_mat[:, i] = bias/no_domains

    return weight_mat, bias_mat

In [12]:
# Initializing the weights and biases

weight1, bias1 = getWeightsAndBiases(3*50*50, 2048, 4)
weight2, bias2 = getWeightsAndBiases(2048, 2048, 4)
weight3, bias3 = getWeightsAndBiases(2048, 1024, 4)
weight4, bias4 = getWeightsAndBiases(1024, 7, 4)

weight1 = weight1.to(device)
weight2 = weight2.to(device)
weight3 = weight3.to(device)
weight4 = weight4.to(device)

bias1 = bias1.to(device)
bias2 = bias2.to(device)
bias3 = bias3.to(device)
bias4 = bias4.to(device)

weight1.requires_grad = True
weight2.requires_grad = True
weight3.requires_grad = True
weight4.requires_grad = True

bias1.requires_grad = True
bias2.requires_grad = True
bias3.requires_grad = True
bias4.requires_grad = True

In [13]:
def MLPModelUndoBias(X, dom_vec) :
    
    x_flattened = X.reshape(-1, 3*50*50)

    # Calculating the current weights and biases by taking inner product
    weight_1_curr = torch.inner(weight1, dom_vec)
    weight_2_curr = torch.inner(weight2, dom_vec)
    weight_3_curr = torch.inner(weight3, dom_vec)
    weight_4_curr = torch.inner(weight4, dom_vec)

    bias_1_curr = torch.inner(bias1, dom_vec)
    bias_2_curr = torch.inner(bias2, dom_vec)
    bias_3_curr = torch.inner(bias3, dom_vec)
    bias_4_curr = torch.inner(bias4, dom_vec)

    out = x_flattened

    out = (x_flattened@(weight_1_curr.T))+bias_1_curr
    out = F.relu(out)
    out = (out@(weight_2_curr.T))+bias_2_curr
    out = F.relu(out)
    out = (out@(weight_3_curr.T))+bias_3_curr
    out = F.relu(out)
    out = (out@(weight_4_curr.T))+bias_4_curr
    out = F.softmax(out, 1)

    return out

In [14]:
epochs = 10
lr = 0.001

# Writing the training loop
for e in range(epochs) :

    running_loss = 0.0
    running_correct = 0
    
    for img, dom, cla in tqdm(train_dataset) :

        img = img.to(device)
        dom = dom.to(device)
        cla = cla.to(device)
        
        # Finding prediction
        pred = MLPModelUndoBias(img.float(), dom.float())

        # Calculating loss
        loss = F.cross_entropy(pred, cla.reshape(-1, 7))

        running_loss += loss

        pred_class = torch.argmax(pred)
        actual_class = torch.argmax(cla)

        if(pred_class == actual_class) :
            running_correct += 1

        loss.backward()

        # Gradient descent
        with torch.no_grad():
            
            weight1 -= weight1.grad * lr
            weight2 -= weight2.grad * lr
            weight3 -= weight3.grad * lr
            weight4 -= weight4.grad * lr

            bias1 -= bias1.grad * lr
            bias2 -= bias2.grad * lr
            bias3 -= bias3.grad * lr
            bias4 -= bias4.grad * lr

            weight1.grad.zero_()
            weight2.grad.zero_()
            weight3.grad.zero_()
            weight4.grad.zero_()

            bias1.grad.zero_()
            bias2.grad.zero_()
            bias3.grad.zero_()
            bias4.grad.zero_()
        
    print(f'Epoch {e}/{epochs-1} | Loss : {running_loss/len(train_dataset)} | Accuracy : {running_correct/len(train_dataset)}')



100%|██████████| 6000/6000 [09:50<00:00, 10.17it/s]


Epoch 0/9 | Loss : 1.9451797008514404 | Accuracy : 0.16133333333333333


100%|██████████| 6000/6000 [09:49<00:00, 10.17it/s]


Epoch 1/9 | Loss : 1.9436601400375366 | Accuracy : 0.17866666666666667


100%|██████████| 6000/6000 [09:50<00:00, 10.16it/s]


...

Epoch 7/9 | Loss : 1.9210090637207031 | Accuracy : 0.218


100%|██████████| 6000/6000 [09:49<00:00, 10.18it/s]


Epoch 8/9 | Loss : 1.9186251163482666 | Accuracy : 0.21616666666666667


100%|██████████| 6000/6000 [09:49<00:00, 10.18it/s]

Epoch 9/9 | Loss : 1.915515661239624 | Accuracy : 0.21566666666666667


In [15]:
# Now extracting the domain agnostic model
def domainAgnosticModel(X) :
  x_flattened = X.reshape(-1, 3*50*50)

  # Calculating the current weights and biases by taking inner product
  weight_1_curr = weight1[:, :, -1]
  weight_2_curr = weight2[:, :, -1]
  weight_3_curr = weight3[:, :, -1]
  weight_4_curr = weight4[:, :, -1]

  bias_1_curr = bias1[:, -1]
  bias_2_curr = bias2[:, -1]
  bias_3_curr = bias3[:, -1]
  bias_4_curr = bias4[:, -1]

  out = x_flattened

  out = (x_flattened@(weight_1_curr.T))+bias_1_curr
  out = F.relu(out)
  out = (out@(weight_2_curr.T))+bias_2_curr
  out = F.relu(out)
  out = (out@(weight_3_curr.T))+bias_3_curr
  out = F.relu(out)
  out = (out@(weight_4_curr.T))+bias_4_curr
  out = F.softmax(out, 1)

  return out

In [16]:
# Testing loop
running_correct = 0
total = 0

for img, dom, cla in tqdm(test_dataloader) :
  img = img.to(device)
  dom = dom.to(device)
  cla = cla.to(device)

  pred = domainAgnosticModel(img.float())

  _, pred_list = torch.max(pred, 1)
  pred_list = pred_list.to(device)

  _, cla_list = torch.max(cla, 1)
  cla_list = cla_list.to(device)

  running_correct += torch.sum(pred_list == cla_list)

print(f'Accuracy of the test dataset : {running_correct/len(test_dataset)}')

100%|██████████| 748/748 [00:10<00:00, 71.64it/s]

Accuracy of the test dataset : 0.21330657601356506
